# 3주차 — 병역학 규칙 기반 경보


## 학습 목표
- 임계치 규칙 설계와 성능 기초 평가

In [ ]:
import sys
import pandas as pd, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import font_manager

# ✅ 1) 한글 폰트 설정 (없으면 영어로 fallback)
candidates = [
    "Malgun Gothic",            # Windows 기본
    "NanumGothic", "Nanum Gothic",
    "Noto Sans CJK KR", "Noto Sans KR",
    "AppleGothic",
    "UnDotum", "Batang", "Gulim"
]
installed = {f.name for f in font_manager.fontManager.ttflist}
choice = next((c for c in candidates if c in installed), None)

if choice:
    mpl.rcParams["font.family"] = choice
    print("✔ Matplotlib 글꼴:", choice)
else:
    print("⚠ 한글 폰트 미탑재 → 제목/레이블은 영어로 대체 권장")

mpl.rcParams["axes.unicode_minus"] = False  # 마이너스 기호 깨짐 방지

# ✅ 2) utils 경로 추가 & 함수 임포트
UTILS_DIR = Path("..") / "utils"
if not UTILS_DIR.exists():
    # 노트북 실행 위치가 다를 수 있으니 대안 경로도 탐색
    alt = Path.cwd().resolve()
    tried = [UTILS_DIR, alt / "utils", alt.parent / "utils"]
    print("utils 탐색:", [str(p) for p in tried])
    for p in tried:
        if (p / "utils.py").exists():
            UTILS_DIR = p
            break
sys.path.append(str(UTILS_DIR))
from utils import risk_index_rulebased

weather = pd.read_csv(str(Path("..")/"data"/"sample_weather_2022.csv"))
weather['date'] = pd.to_datetime(weather['date'])

# 규칙 기반 경보(토이)
risk = weather.apply(risk_index_rulebased, axis=1)
weather['risk_index'] = risk

# 이동합으로 연속 위험일수 개념
weather['risk_7d_sum'] = weather['risk_index'].rolling(7, min_periods=1).sum()

plt.figure(figsize=(10,4))
plt.plot(weather['date'], weather['risk_7d_sum'])
plt.title("7일 위험일수 합")
plt.xlabel("날짜"); plt.ylabel("days")
plt.tight_layout(); plt.show()